<a href="https://colab.research.google.com/github/Yying-Gis/yorkbestliving/blob/main/Traffic_convenience_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install osmnx
import osmnx as ox
!pip install networkx
import networkx as nx
!pip install matplotlib.pyplot
import matplotlib.pyplot as plt
#!pip install folium
#import folium
#!pip install keplergl
#from keplergl import KeplerGl
!pip install geopandas
import geopandas as gpd
#!pip install tqdm
#from tqdm import tqdm
!pip install pandas
import pandas as pd
!pip install RateLimiter
from geopy.extra.rate_limiter import RateLimiter
from geopy.exc import GeocoderTimedOut, GeocoderServiceError

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 1.9 MB/s eta 0:00:00
ERROR: Operation cancelled by user
ERROR: Could not find a version that satisfies the requirement matplotlib.pyplot (from versions: none)
ERROR: No matching distribution found for matplotlib.pyplot
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    retu

In [ ]:
# drive
# osmnx.features.features_from_place(query, tags, *, which_result=None)
drive  = ox.graph_from_place("York, UK", network_type='drive')
#print(type(drive))
  #<class 'networkx.classes.multidigraph.MultiDiGraph'>
    #output (node, edge:include length, travel_time)
drive_projected = ox.project_graph(drive)
nodes_drive, edges_drive = ox.convert.graph_to_gdfs(drive_projected)

fig, ax = ox.plot_graph(drive_projected, show=False, close=False)
edges_drive.plot(ax=ax, linewidth=0.3, edgecolor='blue')
nodes_drive.plot(ax=ax, color='red', markersize=0.7)
plt.show()

KeyboardInterrupt: 

In [ ]:
# add data (footstreet )
footstreets = gpd.read_file("https://1drv.ms/u/c/af33f3905d006110/EYYKXWPdkAFGhEn-Az1FV08BCOs0fH7gtNQb_31cOKjjig?e=KcVTse")

# osmnx.graph.graph_from_place(query, *, network_type='all', simplify=True, retain_all=False, truncate_by_edge=False, which_result=None, custom_filter=None)

# get walking
walk = ox.graph_from_place("York, UK", network_type='walk')

#combine data
# ox.bearings.add_edge_bearings || return : networkx.MultiDiGraph
walk_net = ox.bearing.add_edge_bearings(ox.projection.project_graph(walk, to_crs=footstreets.crs))
walk_projected = ox.project_graph(walk_net)
nodes_walk, edges_walk = ox.convert.graph_to_gdfs(walk_projected)

fig, ax = ox.plot_graph(walk_projected, show=False, close=False)
edges_walk.plot(ax=ax, linewidth=0.3, edgecolor='blue')
nodes_walk.plot(ax=ax, color='red', markersize=0.7)
plt.show()


DataSourceError: '/vsimem/pyogrio_be3500b0b2c8432ebdfede099cf57b6b' not recognized as being in a supported file format.; It might help to specify the correct driver explicitly by prefixing the file path with '<DRIVER>:', e.g. 'CSV:path'.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#get train
train_stations = ox.features_from_place("York, UK", tags={'railway':'station'})
stations_gdf = ox.projection.project_gdf(train_stations.to_crs(walk_projected.graph['crs']))
#print(type(stations_gdf))
  #'geopandas.geodataframe.GeoDataFrame'>
stations_gdf